# Neural Network Enhancement of ORB/RANSAC Outputs

In [1]:
import pandas as pd
import numpy as np
from utils import * # ⚠️⚠️ The functions in utils.py are not correct.
import cv2 as cv
import os
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation # If the module is not found, run `pip install scipy` in this .ipynb in a separate code block.

## Obtaining Training Labels

### Conversion of Net Transformation to Incremental Transformations
⚠️⚠️ The conversion is not correct.

⚠️⚠️ I have not taken into account how the training label given at i is the 'reverse' transformation required to _return_ the image to the original position.

In [2]:
# Read the data, which contains net transformations.
PATH_TO_INPUT_FILE = "train_labels_one_chain_only.csv"
PATH_TO_OUTPUT_FILE = "train_labels_incremental_transformations.csv"

df = pd.read_csv(PATH_TO_INPUT_FILE)

# Create a new CSV file.
with open(PATH_TO_OUTPUT_FILE, "w") as f:

    # Write the first row of data in df to the new CSV file.
    df.iloc[[0]].to_csv(f, index=False)

    # Iterate through the rows of the dataframe, and calculate the incremental transformations.
    for i in range(1, 100):
        prev_row = df.iloc[i-1]
        curr_row = df.iloc[i]

        chain_id = curr_row["chain_id"]

        T_start = np.array([prev_row["x"], prev_row["y"], prev_row["z"]])
        T_end = np.array([curr_row["x"], curr_row["y"], curr_row["z"]])

        R_start = np.array([prev_row["qw"], prev_row["qx"], prev_row["qy"], prev_row["qz"]])
        R_end = np.array([curr_row["qw"], curr_row["qx"], curr_row["qy"], curr_row["qz"]])

        R_incremental, T_incremental = decompose_transformations(R_start, T_start, R_end, T_end)

        # Write the incremental transformations to the new CSV file.
        f.write(f"{chain_id},{i},{T_incremental[0]},{T_incremental[1]},{T_incremental[2]},{R_incremental[0]},{R_incremental[1]},{R_incremental[2]},{R_incremental[3]}\n")

### (Sanity Check) Conversion Back to Net Transformations
Hopefully, the file we generate `PATH_TO_SANITY_CHECK_FILE` has similar transformations as the `PATH_TO_INPUT_FILE`.

Currently, the rotational components are reverse-engineered from the net transformations correctly! 🎉

However, the translational components are not. 🫤

In [6]:
PATH_TO_SANITY_CHECK_FILE = "train_labels_sanity_check.csv"

df = pd.read_csv(PATH_TO_OUTPUT_FILE)

# Create a new CSV file.
with open(PATH_TO_SANITY_CHECK_FILE, "w") as f:

    # Write the first row of data in df to the new CSV file.
    df.iloc[[0]].to_csv(f, index=False)

    # Iterate through the rows of the dataframe, and calculate the net transformation.
    T_net = np.zeros((3))
    R_net = np.array([1, 0, 0, 0])

    for i in range(1, 100):
        curr_row = df.iloc[i]
        chain_id = curr_row["chain_id"]

        T_incremental = np.array([curr_row["x"], curr_row["y"], curr_row["z"]])
        R_incremental = np.array([curr_row["qw"], curr_row["qx"], curr_row["qy"], curr_row["qz"]])

        R_net, T_net = compose_transformations(R_net, T_net, R_incremental, T_incremental)

        # Write the net transformations to the new CSV file.
        f.write(f"{chain_id},{i},{T_net[0]},{T_net[1]},{T_net[2]},{R_net[0]},{R_net[1]},{R_net[2]},{R_net[3]}\n")

# Using ORB, BFMatcher, RANSAC to Compute Transformations
ORB (Oriented FAST and Rotated BRIEF) detects features.

BFMatcher (Brute-Force Matcher) matches features between two images.

RANSAC (RANdom SAmple Consensus) estimates the transformation between two images.

## Image Loading

In [4]:
CHAIN_ID = '0a998b28bd'
ABSOLUTE_PATH_TO_FOLDER = os.path.abspath(os.path.join('..', 'data', 'images', CHAIN_ID))

images = []
for i in range(0, 100):
    PATH_TO_IMAGE = os.path.join(ABSOLUTE_PATH_TO_FOLDER, f"{i:03}" + ".png")
    img = cv.imread(PATH_TO_IMAGE)
    images.append(img)


## Computing Transformations
The transformations $\Delta R_i$ and $\Delta T_i$ are computed between the two images.

We wish to throw these into a neural network to adjust their values, especially for the translational components.

In [5]:
# The camera intrinsic matrix K is given.
# This describes the focal length, optical center, and skew of the camera.
# Source: https://www.drivendata.org/competitions/261/spacecraft-pose-estimation/page/834/#camera-intrinsic-parameters
K = np.array([[5.2125371e+03, 0.0000000e+00, 6.4000000e+02],
              [0.0000000e+00, 6.2550444e+03, 5.1200000e+02],
              [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]])

# For each pair of images (i, i+1),...
for i in range(0, 99):

    ###################################
    # FEATURE DETECTION AND MATCHING  #
    ###################################

    # Create an ORB object.
    orb = cv.ORB_create()
    kp1, des1 = orb.detectAndCompute(images[i], None)
    kp2, des2 = orb.detectAndCompute(images[i+1], None)

    # Create a BFMatcher object.
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply the ratio test to obtain unambiguous (good) matches.
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Draw the matches.
    img_matches = cv.drawMatches(images[i], kp1, images[i+1], kp2, good_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Save the matches in a folder.
    # Create a folder to save the matches if it doesn't already exist.
    # if not os.path.exists("results"):
    #     os.makedirs("results")
    # cv.imwrite("results/matches_{:03d}_{:03d}.png".format(i, i+1), img_matches)

    ###################################
    # TRANSFORMATION CALCULATION      #
    ###################################

    # Extract the matched keypoints.
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Extract the fundamental matrix, which describes the relative motion between two camera angles.
    F, mask = cv.findFundamentalMat(src_pts, dst_pts, cv.FM_RANSAC)

    # Note: Sometimes, the fundamental matrix cannot be found. F will be None, which causes errors later on.
    if F is None:
        print(f"At i = {i:02}, F could not be calculated, so F was set as the identity matrix.")
        F = np.eye(3)

    # Adjust for the effects of the camera's distortion on the image.
    E = K.T @ F @ K

    # Extract the rotation matrix R and translation T.
    _, R, T, mask = cv.recoverPose(E, src_pts, dst_pts, K)
    T = T.flatten() # Change the shape of T from (3, 1) to (3,)

    # Convert the rotation matrix to a quaternion.
    r = Rotation.from_matrix(R)
    q_xyzw = r.as_quat()
    q_wxyz = np.roll(q_xyzw, 1)

    # Print the results.
    # (Turn off scientific notation. Five decimal places. Always leave a space before the ' ' for the sign.)
    np.set_printoptions(suppress=True, precision=5, sign=' ')
    print(f"At i = {i:02}, Change in R = {q_wxyz}, Change in T = {T}")

At i = 00, Change in R = [ 0.97124  0.00115 -0.02021 -0.23724], Change in T = [-0.01174  0.00725  0.9999 ]
At i = 01, Change in R = [ 0.98477 -0.00698  0.0169  -0.17289], Change in T = [-0.08398 -0.04483 -0.99546]
At i = 02, Change in R = [ 0.91657  0.19677  0.09017 -0.33623], Change in T = [-0.00418 -0.00044  0.99999]
At i = 03, Change in R = [ 0.9895  -0.0028  -0.01395  0.14381], Change in T = [ 0.03078 -0.00337 -0.99952]
At i = 04, Change in R = [ 0.97875 -0.0012  -0.00021 -0.20505], Change in T = [ 0.00614  0.0003   0.99998]
At i = 05, Change in R = [ 0.99987 -0.00409  0.00669  0.01383], Change in T = [ 0.00673  0.00576  0.99996]
At i = 06, Change in R = [ 0.40959 -0.23907  0.87764  0.06952], Change in T = [-0.1531  -0.00232  0.98821]
At i = 07, Change in R = [ 0.9981  -0.00068  0.00617  0.06133], Change in T = [ 0.00856  0.00833 -0.99993]
At i = 08, Change in R = [ 0.99992 -0.00136 -0.00608  0.01057], Change in T = [-0.00306  0.00556  0.99998]
At i = 09, Change in R = [ 0.99998 -0